In [ ]:
!/opt/bin/nvidia-smi

Tue Mar 22 08:45:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    30W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!ps -aux|grep python

root          30  0.3  0.0      0     0 ?        Z    08:17   0:05 [python3] <defunct>
root          31  0.0  0.3 160664 41824 ?        S    08:17   0:01 python3 /usr/local/bin/colab-fileshim.py
root          56  0.2  0.4 204852 61444 ?        Sl   08:18   0:03 /usr/bin/python3 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root          73  3.1 17.6 12248752 2349032 ?    Ssl  08:31   0:26 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e34ba27a-6d52-489e-ad86-4b856dc7eb69.json
root          93  0.3  0.1 128672 16584 ?        Sl   08:31   0:02 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 34609 --host 127.0.0.1 --port 18467 --server-access-token a6891e88ba1210801bd4f11228b6ace84456fba2961dbce81cda15e187b9c99b
root         278  0.0  0.0  18380  2996 ?        S    08:31   0:00 bash -c tail -n +0 -F "/root/.config/Google/Dr

In [ ]:
!kill -9 30 31 56 73 93 278 281 512 513

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts"

/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 14.1 MB/s 
     |████████████████████████████████| 6.5 MB 55.1 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import datetime as dt
import pickle

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler, SequentialSampler
from scipy import stats

from transformers import AutoModel, AutoTokenizer, AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_

from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics

from itertools import product

In [ ]:
version = 'v01_32'
os.makedirs(version, exist_ok=True)
os.makedirs('checkpoint', exist_ok=True)

# 데이터 불러오기

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts/klue-sts-v1.1_train.json') ########### 2) 경로 및 파일명 지정 ##########
df['scores'] = df['labels'].map(lambda x: x['real-label'])
df['labels'] = df['labels'].map(lambda x: x['binary-label'])
df = df[['sentence1', 'sentence2', 'labels', 'scores']]
df.head(10)

,sentence1,sentence2,labels,scores
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1,3.714286
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0,0.333333
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0,0.571429
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1,4.714286
5,정부가 새로운 일자리를 직접 창출하는 노력도 배가하겠습니다.,세계에서 우리만큼 오랜 역사와 문화를 공유하는 가까운 이웃이 없습니다.,0,0.000000
6,지하철을 타도 30분안에는 이동이 가능합니다!,"지하철을 탄다고 해도, 30분이면 그곳에 도착할 수 있어요!",1,4.000000
7,사례집은 국립환경과학원 누리집(ecolibrary.me.go.kr)에서 12일부터 ...,주말을 제외한 평일 오후 12시 30분부터 문예회관 공식 페이스북과 유튜브에서는 지...,0,0.000000
8,환퐁기 작동 방법 좀 설명해줘,조명등 낮에 켜놓으면 큰일나,0,0.142857
9,새로운 친구들을 만나고 싶을때 아주 추천합니다.,새로운 친구들을 만나고 싶을 때 추천합니다.,1,4.428571


In [ ]:
# 한국어 불용어 불러오기
import requests 
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

# Korean stop words: 677


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
import re
def remove1(x):
    x =re.sub('[^가-힣 ]','',x)
    x = ' '.join( [w for w in x.split() if w not in stop_words] )

    return x

df['sentence1']= df['sentence1'].apply(remove1)
df['sentence2']= df['sentence2'].apply(remove1)

In [ ]:
def some_func(s1, s2):
    return '[CLS] ' + s1 + ' [SEP] ' + s2 + ' [SEP]'

df['sentence'] = df.apply(lambda x: some_func(s1=x['sentence1'], s2=x['sentence2']), axis=1)

train_df = df[['sentence', 'labels', 'scores']]
train_df.head(3)

,sentence,labels,scores
0,[CLS] 무엇보다도 호스트분들이 너무 친절하셨습니다 [SEP] 무엇보다도 호스트들...,1,4.857143
1,[CLS] 주요 관광지 걸어서 이동가능합니다 [SEP] 위치는 피렌체 중심가까지 걸...,0,1.428571
2,[CLS] 학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위...,0,1.285714


# 설정 및 선언

## 모델, hidden_size 설정

In [ ]:
model_name = 'monologg/kobert'
hidden_size = 768

## Device 설정

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


## Tokenizer 선언

In [ ]:
# KoBERT Tokenizer 파일, sentencepiece 라이브러리 다운로드
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py
!pip install sentencepiece

--2022-03-23 08:19:27--  https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10896 (11K) [text/plain]
Saving to: ‘tokenization_kobert.py’

tokenization_kobert 100%[===================>]  10.64K  --.-KB/s    in 0.001s  

2022-03-23 08:19:27 (8.66 MB/s) - ‘tokenization_kobert.py’ saved [10896/10896]

     |████████████████████████████████| 1.2 MB 13.9 MB/s 


In [ ]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
from transformers import BertModel
bert_embedding_model = BertModel.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

## CustomDataset 선언

In [ ]:
class CustomDataset(Dataset):
    """
  - input_data: list of string
  - target_data: list of int
  """

    def __init__(self, input_data: list, score_data:list, target_data: list) -> None:
        self.X = input_data
        self.Y = score_data
        self.Z = target_data

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (self.X[index], self.Y[index], self.Z[index])


def custom_collate_fn(batch):
    global tokenizer
    
    input_list, score_list, target_list = [_[0] for _ in batch], [_[1] for _ in batch], [_[2] for _ in batch]
    
    tensorized_input = tokenizer(
                                input_list,
                                padding='longest',  # True or 'longest': Pad to the longest sequence in the batch
                                truncation=True,
                                return_tensors='pt',
                                add_special_tokens=False
                                )
    
    tensorized_score = torch.tensor(score_list)
    
    return (tensorized_input, tensorized_score, target_list)

## CustomRegressor 선언

In [ ]:
class CustomRegressor(nn.Module):

    def __init__(self, hidden_size=768):
        super(CustomRegressor, self).__init__()

        self.model = bert_embedding_model

        dropout_rate = 0.1
        linear_layer_hidden_size = 32

    
        self.classifier = nn.Sequential(
                                      nn.Linear(hidden_size, linear_layer_hidden_size), 
                                      nn.ReLU(),
                                      nn.Dropout(dropout_rate),
                                      nn.Linear(linear_layer_hidden_size, 1)
                                      )


    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
    
        outputs = self.model(
                            input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            )
        
        cls_token_last_hidden_states =  outputs['pooler_output']
        
        logits = self.classifier(cls_token_last_hidden_states)
        
        return logits

# 학습 관련 함수

In [ ]:
def initializer(train_dataloader, hidden_size=768, epochs=2, lr=3e-5):

    model = CustomRegressor(hidden_size=hidden_size)

    optimizer = AdamW(model.parameters(), lr=lr)

    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps
                                                )

    return model, optimizer, scheduler



# 모델 저장
def save_checkpoint(path, model, hyperparam_str, optimizer, scheduler, fold_i, epoch, loss):
    file_name = f'{path}/model_{hyperparam_str}_fold{fold_i}.ckpt.epoch{epoch}'

    # torch.save 함수 참고
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss
        }, file_name)

    print(f"Saving epoch {epoch} checkpoint at {file_name}\n")



# input: model, valid_dataloader
# output: loss, acc, recall, precision, f1, pearsonr
def validate(model, valid_dataloader, loss_fct):
   
    # 모델을 evaluate 모드로 설정 & device 할당
    model.eval()
    
    total_loss, total_acc, total_recall, total_precision, total_f1, totel_r = 0, 0, 0, 0, 0, 0
        
    for step, batch in enumerate(valid_dataloader):
        # tensor 연산 전, 각 tensor에 device 할당
        true_labels = [item for item in batch[2]]
        batch = tuple(item.to(device) for item in batch[0:2])
            
        batch_input, batch_label = batch[0], batch[1]

        # gradient 계산하지 않고 forward 진행
        with torch.no_grad():
            logits = model(**batch_input)

        # loss
        loss = loss_fct(logits.squeeze(), batch_label.to(torch.float32))
        total_loss += loss.item()
        
        # 성능측정
        score = logits.squeeze().cpu().numpy()
        pred = (logits.flatten() >= 3).cpu().numpy()
        batch_label = batch_label.cpu().numpy()

        acc = metrics.accuracy_score(pred, true_labels)
        recall = metrics.recall_score(pred, true_labels)
        precision = metrics.precision_score(pred, true_labels)
        f1score = metrics.f1_score(pred, true_labels)
        r = stats.pearsonr(score, batch_label)[0]

        total_acc += acc
        total_recall += recall
        total_precision += precision
        total_f1 += f1score
        totel_r += r

    total_loss /= (step+1)
    total_acc /= (step+1)
    total_recall /= (step+1)
    total_precision /= (step+1)
    total_f1 /= (step+1)
    totel_r /= (step+1)

    return total_loss, total_acc, total_recall, total_precision, total_f1, totel_r



def train(model, train_dataloader, valid_dataloader, optimizer, scheduler, loss_fct, hyperparam_str, fold_i=0, epochs=2, path='v01'):

    # train_dataloaer 학습을 epochs만큼 반복
    epoch_mean_losses = []
    valid_performances = []
    valid_f1_performances = []
    for epoch in range(epochs):
        print(f"Epoch {epoch}---------------")
        
        # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
        total_loss, batch_loss, batch_count = 0,0,0
    
        # model을 train 모드로 설정 & device 할당
        model.train()
        model.to(device)
        
        # data iterator를 돌면서 하나씩 학습
        for step, batch in enumerate(train_dataloader):
            batch_count+=1
            
            # tensor 연산 전, 각 tensor에 device 할당
            batch = tuple(item.to(device) for item in batch[0:2])
            #batch = tuple(item.to(device) for item in batch)

            batch_input, batch_label = batch

            # batch마다 모델이 갖고 있는 기존 gradient를 초기화
            model.zero_grad()

            # forward
            logits = model(**batch_input)

            # loss
            loss = loss_fct(logits.squeeze(), batch_label.to(torch.float32))
            batch_loss += loss.item()
            total_loss += loss.item()

            # backward -> 파라미터의 미분(gradient)를 자동으로 계산
            loss.backward()
            
            # gradient clipping 적용 (max_norm = 1)
            clip_grad_norm_(model.parameters(), max_norm=1)
            
            # optimizer & scheduler 업데이트
            optimizer.step()
            scheduler.step()
            
            # 배치카운트 100 때마다 평균 loss와 lr를 출력
            if (step % 100 == 0 and step != 0):
                learning_rate = optimizer.param_groups[0]['lr']
                print(f"Step : {step}, LR : {learning_rate:.8f}, Avg Loss : {batch_loss / batch_count:.4f}")

                # reset 
                batch_loss, batch_count = 0,0

        # 에폭 에러
        print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
        epoch_mean_losses.append(total_loss/(step+1))
        
        # 검증
        if valid_dataloader is not None:
            print(f'Validation')
            valid_loss, valid_acc, valid_recall, valid_precision, valid_f1, valid_r = validate(model, valid_dataloader, loss_fct)
            print(f"Epoch {epoch}, loss : {valid_loss:.4f}, accuracy : {valid_acc:.4f}, recall : {valid_recall:.4f}, precision : {valid_precision:.4f}, f1 : {valid_f1:.4f}, r_per : {valid_r:.4f}\n")
            valid_performances.append(valid_r)
            valid_f1_performances.append(valid_f1)
        
        # checkpoint 저장
        if (epoch+1) == epochs:  # 마지막 에폭 모델만 저장
            save_checkpoint(path, model, hyperparam_str, optimizer, scheduler, fold_i, epoch, total_loss)

    print("Train Completed. End Program.")
    return valid_performances, epoch_mean_losses, valid_f1_performances

# KFold

In [ ]:
# Cross validation
n_splits = 5
random_state = 42

kfold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
folds = []
for trn_idx, val_idx in kfold.split(train_df):
    folds.append((trn_idx, val_idx))

# 학습

In [ ]:
def train_and_eval_kfold_mean_performance(epochs, lr, batch_size, hyperparam_str):

    print()
    print('=====Training and Evaluating KFold Mean Performance=====')
    print(f"epochs: {epochs}, lr: {lr}, batch_size: {batch_size}")
    print()

    all_fold_per, all_fold_loss, all_fold_per_f1 = [],[],[]

    for fold_i, fold in enumerate(folds):
        print(f'-----------------------------------{fold_i + 1}th fold-----------------------------------------------------')
        train_data = train_df.loc[fold[0]]
        valid_data = train_df.loc[fold[1]]

        # CustomDataset(data: pd.DataFrame, model_name, train_option='train')
        train_dataset = CustomDataset(train_data['sentence'].tolist(), train_data['scores'].tolist(), train_data['labels'].tolist())
        valid_dataset = CustomDataset(valid_data['sentence'].tolist(), valid_data['scores'].tolist(), valid_data['labels'].tolist())

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=custom_collate_fn, shuffle=True)
        valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=custom_collate_fn, shuffle=False)

        model, optimizer, scheduler = initializer(train_dataloader, hidden_size, epochs, lr=lr)
        loss_fct = nn.MSELoss()

        valid_performances, epoch_mean_losses, valid_f1_performances = train(model=model,
                                                                             train_dataloader=train_dataloader,
                                                                             valid_dataloader=valid_dataloader,
                                                                             optimizer=optimizer,
                                                                             scheduler=scheduler,
                                                                             loss_fct=loss_fct,
                                                                             hyperparam_str=hyperparam_str,
                                                                             fold_i=fold_i,
                                                                             epochs=epochs,
                                                                             path=version)
        all_fold_per.append(valid_performances[-1])
        all_fold_loss.append(epoch_mean_losses[-1])
        all_fold_per_f1.append(valid_f1_performances[-1])

        mean_per = np.mean(all_fold_per)
        mean_f1 = np.mean(all_fold_per_f1)

    # 결과 적기  
    with open(f"{version}/{version}_epoch{epochs}_lr{lr}_batchsize{batch_size}.txt", "w") as file:
        file.write(f"------------------------------------- \n")
        file.write(f"time : {str(dt.datetime.today())} \n")
        file.write(f"model_name : {model_name} \n")
        file.write(f"batch_size : {batch_size} \n")
        file.write(f"lr : {lr} \n")
        file.write(f"epoch : {epochs} \n")
        file.write(f"hidden_size : {hidden_size} \n")
        file.write(f"KFold : {n_splits} \n")
        file.write(f"split_random_state : {random_state} \n")
        file.write(f"Kfold Valid 평균 성능 : {mean_per} \n")
        file.write(f"Kfold Valid F1 평균 성능 : {mean_f1}")

    return mean_per, mean_f1

# Hyperparameter Tuning

In [ ]:
# Optimal Hyperparameter Search

epochs = [2, 3, 4]
lr = [2e-5, 3e-5, 5e-5]
batch_size = [32] ########### 3) batch_size 32로 변경 ##########

hyperparam_comb = product(epochs, lr, batch_size)

max_per, max_f1 = 0, 0
max_per_hyperparam, max_f1_hyperparam = None, None

for comb in hyperparam_comb:
    hyperparam_str = '_'.join(list(map(lambda x: str(x), comb)))
    mean_per, mean_f1 = train_and_eval_kfold_mean_performance(*comb,
                                                              hyperparam_str=hyperparam_str)
    if mean_per > max_per:
        max_per = mean_per
        max_per_hyperparam = comb
    if mean_f1 > max_f1:
        max_f1 = mean_f1
        max_f1_hyperparam = comb

print()
print('=====Hyperparameter Tuning Completed=====')
print(f'Best Hyperparams for PearsonR: EPOCHS {max_per_hyperparam[0]}, LR {max_per_hyperparam[1]}, BATCH {max_per_hyperparam[2]}')
print(f'Best PearsonR: {max_per}')
print(f'Best Hyperparams for F1 Score: EPOCHS {max_f1_hyperparam[0]}, LR {max_f1_hyperparam[1]}, BATCH {max_f1_hyperparam[2]}')
print(f'Best F1 Score: {max_f1}')
print()

# Evaluation on Test Dataset

In [ ]:
# Load Test Dataset
df = pd.read_json('/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts/klue-sts-v1.1_dev.json') ########### 4) 경로 및 파일명 지정 ##########
df['scores'] = df['labels'].map(lambda x: x['real-label'])
df['labels'] = df['labels'].map(lambda x: x['binary-label'])
df = df[['sentence1', 'sentence2', 'labels', 'scores']]

# Remove Stop Words
df['sentence1']= df['sentence1'].apply(remove1)
df['sentence2']= df['sentence2'].apply(remove1)

# Concat into One Sentence
df['sentence'] = df.apply(lambda x: some_func(s1=x['sentence1'], s2=x['sentence2']), axis=1)

test_df = df[['sentence', 'labels', 'scores']]

# Create Dataset & Dataloader

test_batch_size = 32

test_dataset = CustomDataset(test_df['sentence'].tolist(), test_df['scores'].tolist(), test_df['labels'].tolist())
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, collate_fn=custom_collate_fn, shuffle=False)

In [ ]:
# Load Model
checkpoint = torch.load('/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts/v01_32/model_4_3e-05_32_fold4.ckpt.epoch3') ########### 5) 경로 및 파일명 지정 ##########

In [ ]:
# Check key types
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])

In [ ]:
# 위에서 구현한 initializer 함수 사용하여 model, optimizer, scheduler 초기화
epochs=1
model, optimizer, scheduler = initializer(test_dataloader,
                                          hidden_size=768,
                                          epochs=epochs,
                                          lr=3e-5)

Total train steps with 1 epochs: 17


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [ ]:
def predict(model, test_dataloader):
    """
    test_dataloader의 label별 확률값과 실제 label 값을 반환
    """

    # model을 eval 모드로 설정 & device 할당
    model.eval()
    model.to(device)

    all_per_logits = []
    all_per_labels = []
    all_f1_logits = []
    all_f1_labels = []

    for step, batch in enumerate(test_dataloader):
        true_labels = [item for item in batch[2]] # binary 정답값
        batch = tuple(item.to(device) for item in batch[0:2])
        batch_input, batch_label = batch
        
        # batch_input을 device 할당
        batch_input.to(device)

        # model에 batch_input을 넣어 logit 반환 & all_logits, all_labels 리스트에 값 추가 
        with torch.no_grad():
          logits = model(**batch_input)

        score = logits.squeeze().cpu().numpy() # real-value 예측값
        pred = (logits.flatten() >= 3).cpu().numpy() # score를 binary 값으로 변환
        batch_label = batch_label.cpu().numpy() # real-value 정답값

        all_per_logits.extend(score)
        all_per_labels.extend(batch_label)
        all_f1_logits.extend(pred)
        all_f1_labels.extend(true_labels)

    return all_per_logits, all_per_labels, all_f1_logits, all_f1_labels

In [ ]:
per_probs, per_labels, f1_probs, f1_labels = predict(model, test_dataloader)

In [ ]:
acc = metrics.accuracy_score(f1_probs, f1_labels)
recall = metrics.recall_score(f1_probs, f1_labels)
precision = metrics.precision_score(f1_probs, f1_labels)
f1_score = metrics.f1_score(f1_probs, f1_labels)
r = stats.pearsonr(per_probs, per_labels)[0]

In [ ]:
# Inference Result

print(f'ACC: {acc:.4f}')
print(f'RECALL: {recall:.4f}')
print(f'PRECISION: {precision:.4f}')
print(f'F1 SCORE: {f1_score:.4f}')
print(f"PEARSON'S R: {r:.4f}")

ACC: 0.8478
RECALL: 0.7809
PRECISION: 0.8909
F1 SCORE: 0.8323
PEARSON'S R: 0.8839


# 문장 유사도 예측 결과 CSV 파일로 저장

In [ ]:
# Save Predicted Result

df = pd.read_csv('/content/drive/MyDrive/Classroom/AI심화과정/test-klue/sts/predicted_dev_set_score.csv') ########### 6) 경로 및 파일명 지정 ##########
df['predict_real_label'] = per_probs
df['predict_binary_label'] = list(map(lambda x:int(x), f1_probs))
display(df.head())

df.to_csv('predicted_dev_set_score_220323.csv') ########### 7) 경로 및 파일명 지정 ##########

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,4.636324,1
1,klue-sts-v1_dev_00001,1.428571,0,1.850857,0
2,klue-sts-v1_dev_00002,1.285714,0,2.177469,0
3,klue-sts-v1_dev_00003,3.714286,1,3.897496,1
4,klue-sts-v1_dev_00004,2.500000,0,3.108995,1
